In [3]:
# max: 3x + y

# 0 ≤ x ≤ 1
# 0 ≤ y ≤ 2
# x + y ≤ 2

# Importe as bibliotecas necessárias
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

# Declare o solucionador.
    # Create the linear solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver('GLOP')


# Crie as variáveis
    # Create the variables x and y.
x = solver.NumVar(0, 1, 'x')
y = solver.NumVar(0, 2, 'y')

print('Number of variables =', solver.NumVariables())


# Defina as restrições,
    # Create a linear constraint, 0 <= x + y <= 2.
ct = solver.Constraint(0, 2, 'ct')
ct.SetCoefficient(x, 1)
ct.SetCoefficient(y, 1)

print('Number of constraints =', solver.NumConstraints())

# Defina a função de objetivo
    # Create the objective function, 3 * x + y.
objective = solver.Objective()
objective.SetCoefficient(x, 3)
objective.SetCoefficient(y, 1)
objective.SetMaximization()

# Invoque o solucionador e exiba os resultados.
solver.Solve()
print('Solution:')
print('Objective value =', objective.Value())
print('x =', x.solution_value())
print('y =', y.solution_value())

Number of variables = 2
Number of constraints = 1
Solution:
Objective value = 4.0
x = 1.0
y = 1.0


In [37]:
from ortools.linear_solver import pywraplp

def solve_pctsp(city_prizes, city_penalties, travel_costs, min_prize):
    # Initialize the solver
    solver = pywraplp.Solver('PCTSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    # Create variables
    n = len(city_prizes)
    x = {}
    for i in range(n):
        for j in range(n):
            x[i, j] = solver.BoolVar(f'x[{i},{j}]')

    y = {}
    for i in range(n):
        y[i] = solver.BoolVar(f'y[{i}]')


    # Create objective function
    objective = solver.Objective()
    for i in range(n):
        for j in range(n):
            if(i!=j):
                objective.SetCoefficient(x[i, j], travel_costs[i][j])
        objective.SetCoefficient(y[i], -1  * city_penalties[i])

    objective.SetOffset(sum(city_penalties))

    objective.SetMinimization()

    # Add constraints
    for i in range(n):
        constraint = solver.RowConstraint(0, 0, f'outgoing_edges_of_{i}')
        for j in range(n):
            if(i!=j):
                constraint.SetCoefficient(x[i, j], 1)
        constraint.SetCoefficient(y[i], -1)

        for j in range(n):
            if(i!=j):
                constraint.SetCoefficient(x[j, i], 1)
        constraint.SetCoefficient(y[i], -1)


    constraint = solver.RowConstraint(min_prize, solver.infinity(), 'minimum_prize')
    for i in range(n):
        constraint.SetCoefficient(y[i], city_prizes[i])


    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        # Extract solution
        tour = []
        i = 0
        while len(tour) < n and sum(city_prizes[t] for t in tour) < min_prize:
            tour.append(i)
            for j in range(n):
                if x[i, j].solution_value() > 0.5:
                    i = j
                    break
        cost = objective.Value()
        return tour, cost
    else:
        return None, None
    

city_prizes = [0, 24, 33, 43, 28]
city_penalties = [1000, 5, 5, 5, 5]
travel_costs = [
    [0, 10, 20, 30, 40],
    [10, 0, 15, 25, 35],
    [20, 15, 0, 18, 28],
    [30, 25, 18, 0, 12],
    [40, 35, 28, 12, 0],
]
min_prize = 60

tour, cost = solve_pctsp(city_prizes, city_penalties, travel_costs, min_prize)

if tour is None:
    print('No solution found.')
else:
    print('Tour:', tour)
    print('Cost:', cost)


KeyError: (-1, 0)

In [17]:
def solve_pctsp(city_prizes, city_penalties, travel_costs, min_prize):
    # Initialize the solver
    solver = pywraplp.Solver('PCTSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    # Create variables
    n = len(city_prizes)
    x = {}
    for i in range(n):
        for j in range(n):
            x[i, j] = solver.BoolVar(f'x[{i},{j}]')

    y = {}
    for i in range(n):
        y[i] = solver.BoolVar(f'y[{i}]')


    # Create objective function
    objective = solver.Objective()
    for i in range(n):
        for j in range(n):
            if(i!=j):
                objective.SetCoefficient(x[i, j], travel_costs[i][j])
        objective.SetCoefficient(y[i], -1  * city_penalties[i])

    objective.SetOffset(sum(city_penalties))

    objective.SetMinimization()

    # Add constraints
    for i in range(n):
        constraint = solver.Constraint(0, 0, f'outgoing_edges_of_{i}')
        for j in range(n):
            if(i!=j):
                constraint.SetCoefficient(x[i, j], 1)
        constraint.SetCoefficient(y[i], -1)

    for i in range(n):
        constraint = solver.Constraint(0, 0, f'incoming_edges_of_{i}')
        for j in range(n):
            if(i!=j):
                constraint.SetCoefficient(x[j, i], 1)
        constraint.SetCoefficient(y[i], -1)

    for i in range(n):
        for j in range(i+1, n):
            constraint = solver.Constraint(-solver.infinity(), 0)
            constraint.SetCoefficient(x[i, j], 1)
            constraint.SetCoefficient(x[j, i], -1)


    constraint = solver.Constraint(min_prize, solver.infinity(), 'minimum_prize')
    for i in range(n):
        constraint.SetCoefficient(y[i], city_prizes[i])


    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
       # Extract solution
        tour = []
        i = 0
        while len(tour) < n and sum(city_prizes[t] for t in tour) < min_prize:
            tour.append(i)
            for j in range(n):
                if x[i, j].solution_value() > 0.5 and j not in tour:
                    i = j
                    break
        cost = objective.Value()
        return tour, cost
    else:
        return None, None

city_prizes = [0, 24, 33, 43, 28]
city_penalties = [1000, 5, 5, 5, 5]
travel_costs = [
    [0, 10, 20, 30, 40],
    [10, 0, 15, 25, 35],
    [20, 15, 0, 18, 28],
    [30, 25, 18, 0, 12],
    [40, 35, 28, 12, 0],
]
min_prize = 60

tour, cost = solve_pctsp(city_prizes, city_penalties, travel_costs, min_prize)

if tour is None:
    print('No solution found.')
else:
    print('Tour:', tour)
    print('Cost:', cost)

Tour: [0, 1, 1, 1]
Cost: 49.0


In [19]:
from ortools.linear_solver import pywraplp

def solve_pctsp(city_prizes, city_penalties, travel_costs, min_prize):
    # Initialize the solver
    solver = pywraplp.Solver('PCTSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    # Create variables
    n = len(city_prizes)
    x = {}
    for i in range(n):
        for j in range(n):
            x[i, j] = solver.BoolVar(f'x[{i},{j}]')

    y = {}
    for i in range(n):
        y[i] = solver.BoolVar(f'y[{i}]')


    # Create objective function
    objective = solver.Objective()
    for i in range(n):
        for j in range(n):
            if(i!=j):
                objective.SetCoefficient(x[i, j], travel_costs[i][j])
        objective.SetCoefficient(y[i], -1  * city_penalties[i])

    objective.SetOffset(sum(city_penalties))

    objective.SetMinimization()

    # Add constraints
    for i in range(n):
        constraint = solver.Constraint(1, 1, f'outgoing_edges_of_{i}')
        for j in range(n):
            if(i!=j):
                constraint.SetCoefficient(x[i, j], 1)

    for i in range(n):
        constraint = solver.Constraint(1, 1, f'incoming_edges_of_{i}')
        for j in range(n):
            if(i!=j):
                constraint.SetCoefficient(x[j, i], 1)

    for i in range(n):
        for j in range(i+1, n):
            constraint = solver.Constraint(0, 1)
            constraint.SetCoefficient(x[i, j], 1)
            constraint.SetCoefficient(x[j, i], 1)

    constraint = solver.Constraint(min_prize, solver.infinity(), 'minimum_prize')
    for i in range(n):
        constraint.SetCoefficient(y[i], city_prizes[i])


    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        # Extract solution
        tour = []
        i = 0
        while len(tour) < n and sum(city_prizes[t] for t in tour) < min_prize:
            tour.append(i)
            for j in range(n):
                if x[i, j].solution_value() >= 0.5 and j not in tour:
                    i = j
                    break
        cost = objective.Value()
        return tour, cost
    else:
        return None, None
    

city_prizes = [0, 24, 33, 43, 28]
city_penalties = [1000, 5, 5, 5, 5]
travel_costs = [
    [0, 10, 20, 30, 40],
    [10, 0, 15, 25, 35],
    [20, 15, 0, 18, 28],
    [30, 25, 18, 0, 12],
    [40, 35, 28, 12, 0],
]
min_prize = 60

tour, cost = solve_pctsp(city_prizes, city_penalties, travel_costs, min_prize)

if tour is None:
    print('No solution found.')
else:
    print('Tour:', tour)
    print('Cost:', cost)

Tour: [0, 3, 4]
Cost: 95.0
